In [1]:
import sys
sys.path.append("/Users/kimsan/Desktop/Lecture-Materials/3-1/AIGS538-Deep-Learning/final-project/")

In [2]:
import torch
import torch.nn as nn
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Model, Wav2Vec2Processor, AutoProcessor
import soundfile as sf
import librosa
from librosa.feature import melspectrogram
from PIL import Image
from uuid import uuid4
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import pandas as pd
import scipy
from tqdm import tqdm

from data.dataset import DataSet

%matplotlib inline

/Users/kimsan/Desktop/Lecture-Materials/3-1/AIGS538-Deep-Learning/final-project/dl-final-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_df = pd.read_csv("/Users/kimsan/Desktop/Lecture-Materials/3-1/AIGS538-Deep-Learning/final-project/data/birdclef-2023/processed/trimed_df.csv")
data_df.head()

,path,label,numeric_label
0,/Users/kimsan/Desktop/Lecture-Materials/3-1/AI...,ruegls1,188
1,/Users/kimsan/Desktop/Lecture-Materials/3-1/AI...,ruegls1,188
2,/Users/kimsan/Desktop/Lecture-Materials/3-1/AI...,ruegls1,188
3,/Users/kimsan/Desktop/Lecture-Materials/3-1/AI...,ruegls1,188
4,/Users/kimsan/Desktop/Lecture-Materials/3-1/AI...,ruegls1,188


In [4]:
class Wav2VecAudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
        
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        
        self.pooling = nn.AvgPool1d(768)

        self.dropout = nn.Dropout(0.5)

        self.projection = nn.Linear(275, 264)

        self.relu = nn.ReLU()
    
    def forward(self, audio):
        processed = self.processor(audio, sampling_rate=16000, return_tensors="pt")["input_values"]
        processed = processed.flatten(0, 1)
        context_representation = self.wav2vec(processed)[0]
        pooling_state = self.pooling(context_representation)
        pooling_state = pooling_state.view(pooling_state.shape[0], pooling_state.shape[2], pooling_state.shape[1])
        logits = self.relu(self.projection(pooling_state))

        return logits

In [5]:
model = Wav2VecAudioClassifier()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.05)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# example_data, sample_rate = librosa.load(data_df.loc[0, "path"])
# example_data_label = data_df.loc[0, "label"]

# trim_data1 = example_data[: sample_rate*3]
# trim_data2 = example_data[3*sample_rate:6*sample_rate]
# batch_data = np.array([trim_data1, trim_data2])

# one_hot_encodings = {label: index for index, label in enumerate(sorted(list(set(data_df["label"]))))}

# example_label_one_hot = torch.zeros(len(set(data_df["label"])))
# example_label_one_hot[one_hot_encodings[example_data_label]] = 1
# data1_one_hot = example_label_one_hot.clone()
# data2_one_hot = example_label_one_hot.clone()

# batched_label = torch.stack((data1_one_hot, data2_one_hot))

# model.train()
# prediction = model.forward(batch_data)
# loss = loss_fn(prediction.flatten(1, 2), batched_label)

# optimizer.zero_grad()
# loss.backward()
# optimizer.step()

In [7]:
params = {
    "batch_size" : 128,
    "shuffle"     : True,
    "num_workers": 6
}
max_epoch = 5

one_hot_encodings = {label: index for index, label in enumerate(sorted(list(set(data_df["label"]))))}

training_set = DataSet(data_df["path"], data_df["label"], one_hot_encodings)
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [8]:
for epoch in range(max_epoch):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Training loop
    model.train()
    for audio, labels in tqdm(training_generator):  # Iterate over your training dataloader
        audio = audio.to("cpu")
        labels = labels.to("cpu")

        optimizer.zero_grad()
        outputs = model.forward(audio)
        loss = loss_fn(outputs.flatten(1, 2), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_labels = torch.max(outputs, dim=1)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    train_loss = total_loss / total_samples
    train_accuracy = total_correct / total_samples

    print(f"epoch: {epoch}, train_loss: {train_loss}, train_accuracy: {train_accuracy}")


  1%|          | 2/260 [13:09<28:16:43, 394.59s/it]